In [ ]:
import sys

import pygame
import math
pygame.init()

# Screen setup
WIDTH, HEIGHT = 800, 500
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Missionaries and Cannibals")

# Colors
GREEN = (34, 139, 34)
BLUE = (0, 191, 255)
BROWN = (139, 69, 19)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (220, 50, 50)
LIGHT_BLUE = (60, 130, 220)
GRAY = (200, 200, 200)
DARK_GRAY = (100, 100, 100)

# Boat settings
BOAT_WIDTH, BOAT_HEIGHT = 100, 40
BOAT_Y = HEIGHT // 2 + 80
boat_side = "left"  # "left" or "right"

# Character settings
CHAR_RADIUS = 20
CHAR_SPACING = 60

# Font
font = pygame.font.SysFont(None, 36)

# Moves counter
moves = 0

# Game state
def reset_game():
    global boat_side, boat_passengers, characters, game_over, win, moves
    boat_side = "left"
    moves = 0
    boat_passengers = []
    characters = []
    # 3 missionaries
    for _ in range(3):
        characters.append({"type": "M", "side": "left", "in_boat": False})
    # 3 cannibals
    for _ in range(3):
        characters.append({"type": "C", "side": "left", "in_boat": False})
    game_over = False
    win = False

reset_game()

# Boat position function
def get_boat_x():
    if boat_side == "left":
        return 250
    else:
        return WIDTH - 250 - BOAT_WIDTH

# Draw triangle for missionary
def draw_missionary(pos):
    x, y = pos
    points = [(x, y - CHAR_RADIUS), (x - CHAR_RADIUS, y + CHAR_RADIUS), (x + CHAR_RADIUS, y + CHAR_RADIUS)]
    pygame.draw.polygon(screen, RED, points)
    label = font.render("M", True, BLACK)
    screen.blit(label, (x - label.get_width() // 2, y - label.get_height() // 2))

# Draw circle for cannibal
def draw_cannibal(pos):
    pygame.draw.circle(screen, LIGHT_BLUE, pos, CHAR_RADIUS)
    label = font.render("C", True, BLACK)
    screen.blit(label, (pos[0] - label.get_width() // 2, pos[1] - label.get_height() // 2))

# Get positions of all characters
def get_positions():
    positions = {}
    # Left side
    left_m = [c for c in characters if c["side"] == "left" and not c["in_boat"] and c["type"] == "M"]
    left_c = [c for c in characters if c["side"] == "left" and not c["in_boat"] and c["type"] == "C"]
    for i, ch in enumerate(left_m):
        positions[id(ch)] = (100, 100 + i * CHAR_SPACING)
    for i, ch in enumerate(left_c):
        positions[id(ch)] = (160, 100 + i * CHAR_SPACING)

    # Right side
    right_m = [c for c in characters if c["side"] == "right" and not c["in_boat"] and c["type"] == "M"]
    right_c = [c for c in characters if c["side"] == "right" and not c["in_boat"] and c["type"] == "C"]
    for i, ch in enumerate(right_m):
        positions[id(ch)] = (WIDTH - 160, 100 + i * CHAR_SPACING)
    for i, ch in enumerate(right_c):
        positions[id(ch)] = (WIDTH - 100, 100 + i * CHAR_SPACING)

    # Boat passengers
    bx = get_boat_x()
    for i, ch in enumerate(boat_passengers):
        positions[id(ch)] = (bx + 30 + i * 40, BOAT_Y - 20)

    return positions

# Check lose condition
def check_lose():
    for side in ["left", "right"]:
        m_count = sum(1 for c in characters if c["side"] == side and c["type"] == "M" and not c["in_boat"])
        c_count = sum(1 for c in characters if c["side"] == side and c["type"] == "C" and not c["in_boat"])
        if m_count > 0 and c_count > m_count:
            return True
    return False

# Check win condition
def check_win():
    return all(c["side"] == "right" for c in characters)

# Draw button
def draw_button(text, x, y, w, h, active=True):
    color = GRAY if active else DARK_GRAY
    pygame.draw.rect(screen, color, (x, y, w, h))
    pygame.draw.rect(screen, BLACK, (x, y, w, h), 2)
    label = font.render(text, True, BLACK)
    screen.blit(label, (x + w // 2 - label.get_width() // 2, y + h // 2 - label.get_height() // 2))

# Draw the game
def draw_game():
    screen.fill(BLUE)
    pygame.draw.rect(screen, GREEN, (0, 0, 200, HEIGHT))
    pygame.draw.rect(screen, GREEN, (WIDTH - 200, 0, 200, HEIGHT))

    # Boat
    boat_x = get_boat_x()
    pygame.draw.rect(screen, BROWN, (boat_x, BOAT_Y, BOAT_WIDTH, BOAT_HEIGHT))

    # Characters
    positions = get_positions()
    for ch in characters:
        pos = positions[id(ch)]
        if ch["type"] == "M":
            draw_missionary(pos)
        else:
            draw_cannibal(pos)

    # Moves counter
    moves_text = font.render(f"Moves: {moves}", True, BLACK)
    screen.blit(moves_text, (WIDTH // 2 - moves_text.get_width() // 2, 20))

    # Sail button
    draw_button("Sail", WIDTH // 2 - 50, HEIGHT - 80, 100, 50, active=len(boat_passengers) > 0)

# Draw game over
def draw_game_over():
    screen.fill(WHITE)
    if win:
        msg = "You Win!"
    else:
        msg = "You Lose!"
    label = font.render(msg, True, BLACK)
    screen.blit(label, (WIDTH // 2 - label.get_width() // 2, HEIGHT // 2 - 50))

    moves_text = font.render(f"Total Moves: {moves}", True, BLACK)
    screen.blit(moves_text, (WIDTH // 2 - moves_text.get_width() // 2, HEIGHT // 2))

    draw_button("Retry", WIDTH // 2 - 50, HEIGHT // 2 + 50, 100, 50)

# Main loop
running = True
while running:
    screen.fill(WHITE)
    if game_over:
        draw_game_over()
    else:
        draw_game()

    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = event.pos
            if not game_over:
                # Sail button
                if WIDTH // 2 - 50 <= mx <= WIDTH // 2 + 50 and HEIGHT - 80 <= my <= HEIGHT - 30:
                    if len(boat_passengers) > 0:
                        # Move boat
                        for ch in boat_passengers:
                            ch["side"] = "right" if boat_side == "left" else "left"
                            ch["in_boat"] = False
                        boat_passengers.clear()
                        boat_side = "right" if boat_side == "left" else "left"
                        moves += 1
                        if check_lose():
                            game_over = True
                            win = False
                        elif check_win():
                            game_over = True
                            win = True
                else:
                    # Character click
                    positions = get_positions()
                    for ch in characters:
                        pos = positions[id(ch)]
                        dist = ((mx - pos[0]) ** 2 + (my - pos[1]) ** 2) ** 0.5
                        if dist <= CHAR_RADIUS:
                            # Same side as boat?
                            if ch["side"] == boat_side:
                                if ch["in_boat"]:
                                    ch["in_boat"] = False
                                    boat_passengers.remove(ch)
                                else:
                                    if len(boat_passengers) < 2:
                                        ch["in_boat"] = True
                                        boat_passengers.append(ch)
                            break
            else:
                # Retry button
                if WIDTH // 2 - 50 <= mx <= WIDTH // 2 + 50 and HEIGHT // 2 + 50 <= my <= HEIGHT // 2 + 100:
                    reset_game()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
